In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/ant-1.7.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

name  version                                             name.1  wmc  \
0    ant      1.7  org.apache.tools.ant.taskdefs.rmic.RmicAdapter...    3   
1    ant      1.7  org.apache.tools.ant.taskdefs.optional.perforc...    5   
2    ant      1.7  org.apache.tools.ant.taskdefs.optional.junit.O...    1   
3    ant      1.7  org.apache.tools.ant.taskdefs.optional.perforc...    8   
4    ant      1.7              org.apache.tools.ant.taskdefs.WaitFor    9   
..   ...      ...                                                ...  ...   
740  ant      1.7              org.apache.tools.ant.taskdefs.Javadoc   92   
741  ant      1.7  org.apache.tools.ant.types.selectors.BaseSelector    6   
742  ant      1.7  org.apache.tools.ant.types.resources.selectors...    7   
743  ant      1.7        org.apache.tools.ant.taskdefs.compilers.Gcj    5   
744  ant      1.7  org.apache.tools.ant.taskdefs.optional.depend....    2   

     dit  noc  cbo  rfc  lcom  ca  ...       dam  moa       mfa       cam  ic  \
0      1    0   10   18     3   1  ...  0.000000    0  0.000000  0.444444   0   
1      2    0    4   13     0   1  ...  1.000000    1  0.700000  0.500000   0   
2      2    0    1    3     0   0  ...  0.000000    0  1.000000  1.000000   0   
3      1    9   13   20    12   9  ...  0.200000    1  0.000000  0.406250   0   
4      3    0    5   26    16   0  ...  1.000000    0  0.800000  0.388889   0   
..   ...  ...  ...  ...   ...  ..  ...       ...  ...       ...       ...  ..   
740    3    0   34  261  3726   8  ...  0.970588   11  0.291339  0.112476   2   
741    3    6   10   10     3   7  ...  1.000000    0  0.857143  0.500000   0   
742    3    5    9   26     0   5  ...  1.000000    0  0.857143  0.314286   1   
743    2    0    8   34     8   1  ...  1.000000    0  0.884615  1.000000   0   
744    1    0    2   18     0   0  ...  1.000000    0  0.000000  0.750000   0   

     cbm        amc  max_cc  avg_cc  bug  
0      0  32.666667       1  0.6667    0  
1      0  13.400000       1  0.6000    0  
2      0   6.000000       0  0.0000    0  
3      0  11.000000       1  0.8750    0  
4      0  19.000000       2  1.0000    1  
..   ...        ...     ...     ...  ...  
740    2  28.021739      15  1.5543    4  
741    0   6.500000       3  1.5000    0  
742    3  19.000000       3  1.5714    0  
743    0  42.000000      11  3.4000    1  
744    0  40.000000       6  3.0000    0  

[745 rows x 24 columns]

In [4]:
train, test = train_test_split(ant_master, test_size=0.2)

In [5]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [6]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3   loc       dam  \
101   14    3    2    8   38    83   2   6    8  0.743590   220  1.000000   
78     1    1    0    3    1     0   3   1    1  2.000000     1  0.000000   
375   11    4    0    7   33    37   0   7    8  0.900000   178  0.428571   
22    11    2    0    7   26     0   5   2   11  0.580000   181  1.000000   
740   92    3    0   34  261  3726   8  34   81  0.960246  2704  0.970588   
..   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   ...       ...   
452    2    3    0    2    3     1   2   0    2  2.000000     7  0.000000   
739    3    4    0    4   11     0   0   4    3  0.000000    53  1.000000   
139    1    4    0    1    3     0   1   0    1  2.000000     6  0.000000   
136    5    1    0   31    5    10  31   0    5  2.000000     5  0.000000   
543    2    2    0    5    4     1   4   1    2  2.000000    10  0.000000   

     moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
101    1  0.740000  0.297619   1    1  14.500000       4  1.2143  
78     0  0.000000  1.000000   0    0   0.000000       1  1.0000  
375    0  0.827586  0.386364   2    3  14.545455       4  1.3636  
22     0  0.529412  0.409091   0    0  15.000000       1  0.7273  
740   11  0.291339  0.112476   2    2  28.021739      15  1.5543  
..   ...       ...       ...  ..  ...        ...     ...     ...  
452    0  0.909091  0.750000   1    1   2.500000       1  0.5000  
739    0  0.962264  0.666667   1    1  16.333333       1  0.6667  
139    0  1.000000  1.000000   0    0   5.000000       0  0.0000  
136    0  0.000000  0.533333   0    0   0.000000       1  1.0000  
543    0  1.000000  0.666667   0    0   4.000000       0  0.0000  

[596 rows x 20 columns]

In [7]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3   loc       dam  \
166   14    1    0   30   96    87   8  23   10  0.884615   905  1.000000   
641    3    5    0    7   14     3   6   2    3  2.000000    52  0.000000   
342   28    1    0   34   63   200  29   7   24  0.805556   528  1.000000   
567    1    6    1    3    2     0   2   1    1  2.000000     4  0.000000   
720   32    2    1   16  140   400   3  14   23  0.970588  1196  0.852941   
..   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   ...       ...   
219    3    1    0    2   21     1   0   2    2  0.500000   120  1.000000   
6     20    1    0    4   40   130   0   4   18  0.736842   345  1.000000   
532   35    1    0    8   73   353   6   2   34  0.807190   776  0.777778   
391   12    3    0   15   47    44   9   8    6  0.781818   240  0.400000   
122    1    7    0    1    7     0   0   1    1  2.000000    40  0.000000   

     moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
166    2  0.000000  0.192308   0    0  63.500000      10  1.7143  
641    0  0.968750  0.555556   1    1  16.333333       3  1.6667  
342    2  0.000000  0.219388   0    0  17.571429       8  1.8571  
567    0  1.000000  1.000000   0    0   3.000000       0  0.0000  
720    2  0.516129  0.218638   1    5  35.312500      28  2.6250  
..   ...       ...       ...  ..  ...        ...     ...     ...  
219    0  0.000000  0.666667   0    0  38.333333       6  3.0000  
6      1  0.000000  0.284211   0    0  15.900000       3  1.1500  
532    0  0.000000  0.160000   1    1  20.657143       4  1.2000  
391    1  0.750000  0.254545   1    3  18.583333       6  2.1667  
122    0  1.000000  1.000000   0    0  37.000000       0  0.0000  

[149 rows x 20 columns]

In [8]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [9]:
y_train

bug
101    2
78     0
375    0
22     0
740    4
..   ...
452    0
739    0
139    0
136    0
543    0

[596 rows x 1 columns]

In [10]:
y_test

bug
166    1
641    0
342    1
567    0
720    1
..   ...
219    0
6      1
532    1
391    0
122    0

[149 rows x 1 columns]

In [11]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [12]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [16]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [17]:
y = y_test['bug'].tolist()
y

[1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 6,
 0,
 0,
 1,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 4,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 4,
 0,
 1,
 1,
 0,
 0]

In [18]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(10):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_ant1.7/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
6/6 [==============================] - 1s 24ms/step - loss: 23887.0527 - mse: 23887.0527 - mae: 56.9108 - root_mean_squared_error: 154.5544 - mean_squared_logarithmic_error: 9.5979
Epoch 2/100
6/6 [==============================] - 0s 4ms/step - loss: 22856.3125 - mse: 22856.3125 - mae: 55.2800 - root_mean_squared_error: 151.1830 - mean_squared_logarithmic_error: 9.4029
Epoch 3/100
6/6 [==============================] - 0s 4ms/step - loss: 22047.2949 - mse: 22047.2949 - mae: 53.6555 - root_mean_squared_error: 148.4833 - mean_squared_logarithmic_error: 9.2068
Epoch 4/100
6/6 [==============================] - 0s 4ms/step - loss: 20952.4336 - mse: 20952.4336 - mae: 52.0543 - root_mean_squared_error: 144.7496 - mean_squared_logarithmic_error: 9.0162
Epoch 5/100
6/6 [==============================] - 0s 4ms/step - loss: 20186.2148 - mse: 20186.2148 - mae: 50.5661 - root_mean_squared_error: 142.0782 - mean_squared_logarithmic_error: 8.8185
Epoch 6/100
6/6 [============

6/6 [==============================] - 0s 4ms/step - loss: 3678.2493 - mse: 3678.2493 - mae: 11.2467 - root_mean_squared_error: 60.6486 - mean_squared_logarithmic_error: 1.7800
Epoch 43/100
6/6 [==============================] - 0s 4ms/step - loss: 3479.7896 - mse: 3479.7896 - mae: 10.7559 - root_mean_squared_error: 58.9897 - mean_squared_logarithmic_error: 1.7097
Epoch 44/100
6/6 [==============================] - 0s 4ms/step - loss: 3343.3379 - mse: 3343.3379 - mae: 10.3610 - root_mean_squared_error: 57.8216 - mean_squared_logarithmic_error: 1.6554
Epoch 45/100
6/6 [==============================] - 0s 4ms/step - loss: 3165.3840 - mse: 3165.3840 - mae: 9.9917 - root_mean_squared_error: 56.2617 - mean_squared_logarithmic_error: 1.6037
Epoch 46/100
6/6 [==============================] - 0s 4ms/step - loss: 3013.2905 - mse: 3013.2905 - mae: 9.6278 - root_mean_squared_error: 54.8934 - mean_squared_logarithmic_error: 1.5553
Epoch 47/100
6/6 [==============================] - 0s 3ms/step -

6/6 [==============================] - 0s 4ms/step - loss: 487.3419 - mse: 487.3419 - mae: 2.4167 - root_mean_squared_error: 22.0758 - mean_squared_logarithmic_error: 0.5147
Epoch 86/100
6/6 [==============================] - 0s 3ms/step - loss: 468.1272 - mse: 468.1272 - mae: 2.3467 - root_mean_squared_error: 21.6362 - mean_squared_logarithmic_error: 0.5031
Epoch 87/100
6/6 [==============================] - 0s 4ms/step - loss: 443.1688 - mse: 443.1688 - mae: 2.2748 - root_mean_squared_error: 21.0516 - mean_squared_logarithmic_error: 0.4923
Epoch 88/100
6/6 [==============================] - 0s 4ms/step - loss: 421.8150 - mse: 421.8150 - mae: 2.2057 - root_mean_squared_error: 20.5381 - mean_squared_logarithmic_error: 0.4826
Epoch 89/100
6/6 [==============================] - 0s 4ms/step - loss: 403.7533 - mse: 403.7533 - mae: 2.1536 - root_mean_squared_error: 20.0936 - mean_squared_logarithmic_error: 0.4751
Epoch 90/100
6/6 [==============================] - 0s 3ms/step - loss: 388.82

6/6 [==============================] - 0s 3ms/step - loss: 1.5403 - mse: 1.5403 - mae: 0.4497 - root_mean_squared_error: 1.2411 - mean_squared_logarithmic_error: 0.2649
Epoch 28/100
6/6 [==============================] - 0s 3ms/step - loss: 1.5403 - mse: 1.5403 - mae: 0.4497 - root_mean_squared_error: 1.2411 - mean_squared_logarithmic_error: 0.2649
Epoch 29/100
6/6 [==============================] - 0s 3ms/step - loss: 1.5403 - mse: 1.5403 - mae: 0.4497 - root_mean_squared_error: 1.2411 - mean_squared_logarithmic_error: 0.2649
Epoch 30/100
6/6 [==============================] - 0s 4ms/step - loss: 1.5403 - mse: 1.5403 - mae: 0.4497 - root_mean_squared_error: 1.2411 - mean_squared_logarithmic_error: 0.2649
Epoch 31/100
6/6 [==============================] - 0s 4ms/step - loss: 1.5403 - mse: 1.5403 - mae: 0.4497 - root_mean_squared_error: 1.2411 - mean_squared_logarithmic_error: 0.2649
Epoch 32/100
6/6 [==============================] - 0s 3ms/step - loss: 1.5403 - mse: 1.5403 - mae: 0.4

6/6 [==============================] - 0s 3ms/step - loss: 1.5403 - mse: 1.5403 - mae: 0.4497 - root_mean_squared_error: 1.2411 - mean_squared_logarithmic_error: 0.2649
Epoch 73/100
6/6 [==============================] - 0s 4ms/step - loss: 1.5403 - mse: 1.5403 - mae: 0.4497 - root_mean_squared_error: 1.2411 - mean_squared_logarithmic_error: 0.2649
Epoch 74/100
6/6 [==============================] - 0s 3ms/step - loss: 1.5403 - mse: 1.5403 - mae: 0.4497 - root_mean_squared_error: 1.2411 - mean_squared_logarithmic_error: 0.2649
Epoch 75/100
6/6 [==============================] - 0s 3ms/step - loss: 1.5403 - mse: 1.5403 - mae: 0.4497 - root_mean_squared_error: 1.2411 - mean_squared_logarithmic_error: 0.2649
Epoch 76/100
6/6 [==============================] - 0s 4ms/step - loss: 1.5403 - mse: 1.5403 - mae: 0.4497 - root_mean_squared_error: 1.2411 - mean_squared_logarithmic_error: 0.2649
Epoch 77/100
6/6 [==============================] - 0s 4ms/step - loss: 1.5403 - mse: 1.5403 - mae: 0.4

...

6/6 [==============================] - 0s 3ms/step - loss: 26541.3359 - mse: 26541.3398 - mae: 94.3964 - root_mean_squared_error: 162.9151 - mean_squared_logarithmic_error: 14.0096
Epoch 18/100
6/6 [==============================] - 0s 4ms/step - loss: 25508.6465 - mse: 25508.6465 - mae: 92.5830 - root_mean_squared_error: 159.7143 - mean_squared_logarithmic_error: 13.8767
Epoch 19/100
6/6 [==============================] - 0s 3ms/step - loss: 24643.7324 - mse: 24643.7324 - mae: 90.8822 - root_mean_squared_error: 156.9832 - mean_squared_logarithmic_error: 13.7438
Epoch 20/100
6/6 [==============================] - 0s 3ms/step - loss: 23769.2012 - mse: 23769.2012 - mae: 89.1766 - root_mean_squared_error: 154.1726 - mean_squared_logarithmic_error: 13.6090
Epoch 21/100
6/6 [==============================] - 0s 3ms/step - loss: 22926.9941 - mse: 22926.9941 - mae: 87.4640 - root_mean_squared_error: 151.4166 - mean_squared_logarithmic_error: 13.4715
Epoch 22/100
6/6 [=========================

6/6 [==============================] - 0s 3ms/step - loss: 6438.3511 - mse: 6438.3516 - mae: 43.0874 - root_mean_squared_error: 80.2393 - mean_squared_logarithmic_error: 8.8049
Epoch 59/100
6/6 [==============================] - 0s 3ms/step - loss: 6230.2202 - mse: 6230.2202 - mae: 42.2226 - root_mean_squared_error: 78.9317 - mean_squared_logarithmic_error: 8.6788
Epoch 60/100
6/6 [==============================] - 0s 4ms/step - loss: 6013.8472 - mse: 6013.8472 - mae: 41.3525 - root_mean_squared_error: 77.5490 - mean_squared_logarithmic_error: 8.5549
Epoch 61/100
6/6 [==============================] - 0s 4ms/step - loss: 5820.9448 - mse: 5820.9448 - mae: 40.5353 - root_mean_squared_error: 76.2951 - mean_squared_logarithmic_error: 8.4330
Epoch 62/100
6/6 [==============================] - 0s 3ms/step - loss: 5645.3433 - mse: 5645.3433 - mae: 39.7122 - root_mean_squared_error: 75.1355 - mean_squared_logarithmic_error: 8.3092
Epoch 63/100
6/6 [==============================] - 0s 4ms/step

5/5 [==============================] - 0s 3ms/step - loss: 6289.7666 - mse: 6289.7666 - mae: 26.4372 - root_mean_squared_error: 79.3081 - mean_squared_logarithmic_error: 4.6810
score of 8th run is: 6289.7666015625, 6289.7666015625, 26.437213897705078, 79.30805206298828, 4.680956840515137
Run no: 9
Epoch 1/100
6/6 [==============================] - 1s 16ms/step - loss: 1339.3009 - mse: 1339.3009 - mae: 23.3284 - root_mean_squared_error: 36.5965 - mean_squared_logarithmic_error: 6.9669
Epoch 2/100
6/6 [==============================] - 0s 3ms/step - loss: 1125.3999 - mse: 1125.3999 - mae: 21.3936 - root_mean_squared_error: 33.5470 - mean_squared_logarithmic_error: 6.5924
Epoch 3/100
6/6 [==============================] - 0s 3ms/step - loss: 940.9159 - mse: 940.9159 - mae: 19.5277 - root_mean_squared_error: 30.6744 - mean_squared_logarithmic_error: 6.2202
Epoch 4/100
6/6 [==============================] - 0s 3ms/step - loss: 767.3171 - mse: 767.3171 - mae: 17.6450 - root_mean_squared_erro

Epoch 43/100
6/6 [==============================] - 0s 3ms/step - loss: 3.4224 - mse: 3.4224 - mae: 0.8972 - root_mean_squared_error: 1.8500 - mean_squared_logarithmic_error: 0.4695
Epoch 44/100
6/6 [==============================] - 0s 3ms/step - loss: 3.3278 - mse: 3.3278 - mae: 0.8800 - root_mean_squared_error: 1.8242 - mean_squared_logarithmic_error: 0.4604
Epoch 45/100
6/6 [==============================] - 0s 4ms/step - loss: 3.2337 - mse: 3.2337 - mae: 0.8641 - root_mean_squared_error: 1.7982 - mean_squared_logarithmic_error: 0.4528
Epoch 46/100
6/6 [==============================] - 0s 4ms/step - loss: 3.1271 - mse: 3.1271 - mae: 0.8454 - root_mean_squared_error: 1.7684 - mean_squared_logarithmic_error: 0.4443
Epoch 47/100
6/6 [==============================] - 0s 3ms/step - loss: 3.0404 - mse: 3.0404 - mae: 0.8306 - root_mean_squared_error: 1.7437 - mean_squared_logarithmic_error: 0.4377
Epoch 48/100
6/6 [==============================] - 0s 3ms/step - loss: 2.9690 - mse: 2.96

6/6 [==============================] - 0s 3ms/step - loss: 2.0333 - mse: 2.0333 - mae: 0.5957 - root_mean_squared_error: 1.4260 - mean_squared_logarithmic_error: 0.3326
Epoch 89/100
6/6 [==============================] - 0s 4ms/step - loss: 2.0290 - mse: 2.0290 - mae: 0.5940 - root_mean_squared_error: 1.4244 - mean_squared_logarithmic_error: 0.3320
Epoch 90/100
6/6 [==============================] - 0s 3ms/step - loss: 2.0186 - mse: 2.0186 - mae: 0.5917 - root_mean_squared_error: 1.4208 - mean_squared_logarithmic_error: 0.3308
Epoch 91/100
6/6 [==============================] - 0s 3ms/step - loss: 2.0102 - mse: 2.0102 - mae: 0.5897 - root_mean_squared_error: 1.4178 - mean_squared_logarithmic_error: 0.3298
Epoch 92/100
6/6 [==============================] - 0s 4ms/step - loss: 2.0025 - mse: 2.0025 - mae: 0.5879 - root_mean_squared_error: 1.4151 - mean_squared_logarithmic_error: 0.3289
Epoch 93/100
6/6 [==============================] - 0s 4ms/step - loss: 1.9956 - mse: 1.9956 - mae: 0.5

In [19]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_ant1.7/'

In [20]:
total_training_time

[4.873072624206543,
 4.0540244579315186,
 4.069300413131714,
 3.6613335609436035,
 3.8450167179107666,
 4.09345269203186,
 4.180105686187744,
 4.2217116355896,
 3.8637025356292725,
 4.108536005020142]

In [21]:
print("avg training time: " + str(sum(total_training_time)/10))

avg training time: 4.097025632858276


In [22]:
total_training_loss_history

[{'loss': [23887.052734375,
   22856.3125,
   22047.294921875,
   20952.43359375,
   20186.21484375,
   19340.427734375,
   18534.765625,
   17682.904296875,
   16956.58203125,
   16224.5146484375,
   15595.17578125,
   14824.3125,
   14157.3505859375,
   13540.6640625,
   13011.8486328125,
   12377.3505859375,
   11794.12109375,
   11239.28125,
   10743.11328125,
   10298.1337890625,
   9837.005859375,
   9371.40625,
   8978.96484375,
   8483.0029296875,
   8163.23583984375,
   7769.9716796875,
   7410.5791015625,
   7056.15771484375,
   6731.28857421875,
   6498.349609375,
   6171.4892578125,
   5886.5849609375,
   5567.0634765625,
   5350.2099609375,
   5077.478515625,
   4835.87646484375,
   4643.40283203125,
   4411.62109375,
   4240.0302734375,
   4041.408935546875,
   3846.194580078125,
   3678.249267578125,
   3479.78955078125,
   3343.337890625,
   3165.384033203125,
   3013.29052734375,
   2869.921142578125,
   2739.44384765625,
   2666.41748046875,
   2486.611083984375,
   2

In [23]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(10):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/10))
print("avg mae of training: " + str(sum_mae/10))

avg mse of training: 262.5672446846962
avg mae of training: 2.564371779561043


In [24]:
total_scores

[[6.637182712554932,
  6.637182712554932,
  0.7989383935928345,
  2.576272964477539,
  0.40183714032173157],
 [1.3422819375991821,
  1.3422819375991821,
  0.46979865431785583,
  1.1585688591003418,
  0.2704995572566986],
 [120.17777252197266,
  120.17777252197266,
  1.371564269065857,
  10.962562561035156,
  0.30970314145088196],
 [1.3422819375991821,
  1.3422819375991821,
  0.46979865431785583,
  1.1585688591003418,
  0.2704995572566986],
 [35.465999603271484,
  35.465999603271484,
  1.2604091167449951,
  5.955333709716797,
  0.4923323690891266],
 [1.8973079919815063,
  1.8973079919815063,
  0.5456903576850891,
  1.3774280548095703,
  0.3161693215370178],
 [1.3422819375991821,
  1.3422819375991821,
  0.46979865431785583,
  1.1585688591003418,
  0.2704995572566986],
 [7.2376389503479,
  7.2376389503479,
  0.8937824368476868,
  2.6902859210968018,
  0.3479672372341156],
 [6289.7666015625,
  6289.7666015625,
  26.437213897705078,
  79.30805206298828,
  4.680956840515137],
 [1.36752319335

In [25]:
test_mae = []
for i in range(10):
    test_mae.append(total_scores[i][2])

In [26]:
test_mae

[0.7989383935928345,
 0.46979865431785583,
 1.371564269065857,
 0.46979865431785583,
 1.2604091167449951,
 0.5456903576850891,
 0.46979865431785583,
 0.8937824368476868,
 26.437213897705078,
 0.5091630816459656]

In [27]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(10):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/10))
print("avg mae: " + str(sum_mae/10))
print("avg are: " + str(sum_are/10))

avg mse: 646.6576872348785
avg mae: 3.3226157516241073
avg are: 1.3406599402427672


In [28]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 3.4186768531799316
median testing mse: 4.267245352268219


In [29]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.5786158442497253
median testing mae: 0.6723143756389618


In [30]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.2902684807777405


In [31]:
test_ARE

[0.31879196,
 0.26174498,
 0.4261745,
 0.26174498,
 0.4161074,
 0.26174498,
 0.26174498,
 0.4043624,
 10.532438,
 0.26174498]